### json 불러와서 저장하기

In [77]:
import numpy as np
import pandas as pd
import json
import jpype
import glob
from random import shuffle
from bs4 import BeautifulSoup as bs

#data_path = "data"
data_path = "data(test)"

file_list = glob.glob("%s/*.json" % data_path)

shuffle(file_list)

json_train=[]

for json_file_name in file_list:
    json_file = json.loads(open(json_file_name).read())
    json_train += json_file["articles"]

### json에서 title 형태소 추출

In [78]:
from konlpy.tag import Mecab

mecab = Mecab()

labeled_train = []

for cnt, article in enumerate(json_train):
    
    if cnt % 10000 == 0:
        print cnt

    labeled_train.append({
            "istroll": article["is_troll"],
            "title_pos": ["%s_%s" % (word, pos) for word, pos in mecab.pos(article["title"])]
        })

labeled_train = pd.DataFrame.from_dict(labeled_train)
labeled_train

0


,istroll,title_pos
0,False,"[일간_NNG, 워스트_NNP, 마을_NNG, 회관_NNG, 입니다_VCP+EC]"
1,False,"[일간_NNG, 워스트_NNP, 일_NNG, 워_NNG, 입니다_VCP+EC]"
2,False,"[오_IC, !_SF]"
3,False,"[1_SN, 등_NNB]"
4,False,"[여기_NP, 가_JKS, 수용소_NNG, 냐_VCP+EF, ?_SF]"
5,False,"[여기_NP, 가_JKS, 일_NR, 워_IC, !_SF]"
6,False,"[지_MM, 젼_NNG]"
7,False,"[함수_NNG, 니_VCP+EC, +_SY, 빛돌_NNP, 이_JKS, 합동_NNG..."
8,False,"[캬_IC, ._SF, .._SY, 미루_VV, 고_EC, 미루_VV, 다_EC, ..."
9,False,"[일간_NNG, ㅋㅋㅋㅋㅋㅋㅋ워슼ㅋㅋㅋㅋㅋ틐ㅋㅋㅋㅋㅋㅋ_UNKNOWN]"


###bag of words

In [79]:
from sklearn.feature_extraction.text import CountVectorizer

def make_bag_of_words(labeled_train, max_features, col_name):
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                                stop_words = None, max_features=max_features)
    
    sentences = labeled_train[col_name].apply(lambda x:" ".join(x))
    train_data_features = vectorizer.fit_transform(sentences).toarray()

    col = ["bow_%s_%s" % (col_name, data) for data in vectorizer.get_feature_names()]
    #col = ["bow%s%s" % (col_name, str(i)) for  in range(0, max_features)]
    df_bow = pd.DataFrame(train_data_features, columns = col, index=labeled_train.index)
    
    labeled_train = pd.concat([labeled_train, df_bow],axis=1)
    
    return labeled_train

##LDA

In [80]:
import gensim
from gensim import corpora, models
from gensim.corpora import TextCorpus, MmCorpus, Dictionary
from nltk.corpus import stopwords
from optparse import OptionParser

def make_lda(train, keep_n, num_topics, col_name):

    data = train[col_name]

    dictionary = corpora.Dictionary(data)
    dictionary.filter_extremes(keep_n=keep_n)

    corpus = [dictionary.doc2bow(text) for text in data]

    print("Make Lda..")

#    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1,distributed = True)
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1)
    
    num = len(train)    
    df = []
    for i in range(0,num):
        if i%10000 == 0: print(i)
        temp = [i[1] for i in lda.get_document_topics(corpus[i],minimum_probability=0)]
        #temp = max(lda.get_document_topics(corpus[i]),key=lambda item:item[1])[0]
        df.append(temp)
    
    col = ["lda_%s_%d" % (col_name, data) for data in range(0, num_topics)]
    df = pd.DataFrame(df, columns = col)
    df.index = index
    
    train = pd.concat([train,df],axis=1)
        
    return train

In [81]:
max_features = 300
#%time labeled_train = make_lda(labeled_train, 5000, 1000, "title_pos")
%time labeled_train = make_bag_of_words(labeled_train, 300, "title_pos")

CPU times: user 8.09 ms, sys: 5.04 ms, total: 13.1 ms
Wall time: 13.8 ms


### predictor, model 생성

In [74]:
from sklearn.ensemble import RandomForestClassifier

label = 'istroll'
pre = labeled_train.columns.drop(['title_pos', label])

model = RandomForestClassifier(n_estimators=10, n_jobs=6)

###cross validation

In [75]:
from sklearn import cross_validation

cv_value = 3

scores = cross_validation.cross_val_score(model, labeled_train[pre], labeled_train[label], cv=cv_value, scoring="roc_auc")
cv_result = scores.mean()

print(cv_result)

0.647444316789


##Result
  * title의 morphs를 bag of words로 변환 (feature : 1,000개) - 0.612752193303
  * title의 pos를 bag of words로 변환 (feature : 1,000개) - 0.630053006037
  * 데이터를 셔플하고 title의 pos를 bag of words로 변환 (feature : 1,000개) - 0.658894903487
  
  * title의 pos를 LDA로 변환 (keep_n : 5,000, num_topics : 1,000) - 0.592446285012

In [76]:
import pickle

pickle.dump(labeled_train[pre], open("title_%d.p" % max_features, "wb"), protocol = pickle.HIGHEST_PROTOCOL)